In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.options.display.max_columns=1000 
pd.options.display.max_rows=1000
import numpy as np

In [2]:
import sys
sys.path.append('ml_modules/')

In [3]:
DATA_DIR = 'C:/Users/arvin/Springboard/Capstone-3/AMexpert/train_AUpWtIz/'
DW_OUTPUTS='C:/Users/arvin/Springboard/Capstone-3/AMexpert/train_AUpWtIz/DW_OUTPUTS/'

train = pd.read_csv(DATA_DIR + 'train.csv')
"train", train.shape

test = pd.read_csv(DATA_DIR + 'test_QyjYwdj.csv')
"test", test.shape

campaign_data = pd.read_csv(DATA_DIR + 'campaign_data.csv')
campaign_data['start_date'] = pd.to_datetime(campaign_data.start_date, format='%d/%m/%y')
campaign_data['end_date'] = pd.to_datetime(campaign_data.end_date,  format='%d/%m/%y')
campaign_data['duration'] = (campaign_data.end_date - campaign_data.start_date).dt.days

('train', (78369, 5))

('test', (50226, 4))

Merge Features

In [4]:
cust_hist_trans = pd.read_csv(DW_OUTPUTS +'cust_hist_trans.csv')
cust_hist_trans['start_date'] = pd.to_datetime(cust_hist_trans.start_date)

cust_hist_trans_daily = pd.read_csv(DW_OUTPUTS +'cust_hist_trans_daily.csv')
cust_hist_trans_daily['start_date'] = pd.to_datetime(cust_hist_trans_daily.start_date)

coup_hist_trans = pd.read_csv(DW_OUTPUTS +'coup_hist_trans.csv')
coup_hist_trans['start_date'] = pd.to_datetime(coup_hist_trans.start_date)

coup_hist_trans_daily = pd.read_csv(DW_OUTPUTS +'coup_hist_trans_daily.csv')
coup_hist_trans_daily['start_date'] = pd.to_datetime(coup_hist_trans_daily.start_date)

cust_coup_hist_trans = pd.read_csv(DW_OUTPUTS +'cust_coup_hist_trans.csv')
cust_coup_hist_trans['start_date'] = pd.to_datetime(cust_coup_hist_trans.start_date)

cust_coup_hist_trans_daily = pd.read_csv(DW_OUTPUTS +'cust_coup_hist_trans_daily.csv')
cust_coup_hist_trans_daily['start_date'] = pd.to_datetime(cust_coup_hist_trans_daily.start_date)

coup_brand_hist_trans = pd.read_csv(DW_OUTPUTS +'coup_brand_hist_trans.csv')
coup_brand_hist_trans['start_date'] = pd.to_datetime(coup_brand_hist_trans.start_date)

cust_brand_hist_trans = pd.read_csv(DW_OUTPUTS +'cust_brand_hist_trans.csv')
cust_brand_hist_trans['start_date'] = pd.to_datetime(cust_brand_hist_trans.start_date)

cust_coup_brand_hist_trans = pd.read_csv(DW_OUTPUTS +'cust_coup_brand_hist_trans.csv')
cust_coup_brand_hist_trans['start_date'] = pd.to_datetime(cust_coup_brand_hist_trans.start_date)

In [5]:
def merge_dfs(df):
    df = df.merge(campaign_data, on='campaign_id', how='left', suffixes=['', '_camp']
                  
    ).merge(coup_hist_trans, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_coht']
    ).merge(coup_hist_trans_daily, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_cohtd']
            
    ).merge(cust_hist_trans, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cuht']
    ).merge(cust_hist_trans_daily, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cuhtd']
            
    ).merge(cust_coup_hist_trans, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucoht']
    ).merge(cust_coup_hist_trans_daily, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucohtd']
            
    ).merge(coup_brand_hist_trans, on=['coupon_id', 'start_date'], how='left', suffixes=['', '_cobrht']
    ).merge(cust_brand_hist_trans, on=['customer_id', 'start_date'], how='left', suffixes=['', '_cubrht']
    ).merge(cust_coup_brand_hist_trans, on=['customer_id', 'coupon_id','start_date'], how='left', suffixes=['', '_cucobrht']
            
    )
    df.shape
    df.head()
    return df
train.shape, test.shape
train = merge_dfs(train)
test = merge_dfs(test)

train.shape, test.shape

((78369, 5), (50226, 4))

((78369, 126), (50226, 125))

In [6]:
train.columns.tolist()

['id',
 'campaign_id',
 'coupon_id',
 'customer_id',
 'redemption_status',
 'campaign_type',
 'start_date',
 'end_date',
 'duration',
 'item_id_count',
 'selling_price_sum',
 'selling_price_mean',
 'selling_price_std',
 'other_discount_sum',
 'other_discount_mean',
 'coupon_discount_sum',
 'coupon_discount_mean',
 'coupon_applied_sum',
 'coupon_applied_mean',
 'date_count',
 'selling_price_mean_cohtd',
 'selling_price_std_cohtd',
 'selling_price_last',
 'other_discount_mean_cohtd',
 'other_discount_std',
 'other_discount_last',
 'coupon_discount_mean_cohtd',
 'coupon_discount_std',
 'coupon_discount_last',
 'coupon_applied_mean_cohtd',
 'coupon_applied_std',
 'coupon_applied_last',
 'selling_price_ewm_mean',
 'selling_price_ewm_std',
 'coupon_discount_ewm_mean',
 'coupon_discount_ewm_std',
 'coupon_applied_ewm_mean',
 'coupon_applied_ewm_std',
 'item_id_count_cuht',
 'selling_price_sum_cuht',
 'selling_price_mean_cuht',
 'selling_price_std_cuht',
 'other_discount_sum_cuht',
 'other_dis

In [7]:
id_col = 'id'
target_col = 'redemption_status'

columns_to_drop = []
columns_to_drop = [c for c in columns_to_drop if c in train.columns]

cat_cols = [
    'campaign_id',
    'coupon_id',
    'customer_id',
    'campaign_type',
    'start_date',
    'end_date',    
]
cat_cols = [c for c in cat_cols if c in train.columns]
cat_cols

['campaign_id',
 'coupon_id',
 'customer_id',
 'campaign_type',
 'start_date',
 'end_date']

In [8]:
print(id_col)
print(target_col)
print(columns_to_drop)
print(cat_cols)

id
redemption_status
[]
['campaign_id', 'coupon_id', 'customer_id', 'campaign_type', 'start_date', 'end_date']


In [9]:
# imputing categorical columns
train[cat_cols] = train[cat_cols].fillna('Missing')
test[cat_cols] = test[cat_cols].fillna('Missing')

# imputing numerical columns
train = train.fillna(-1)
test = test.fillna(-1)

In [10]:
group_col = train['campaign_id'].astype(str) 
group_col_coupon = train['coupon_id'].astype(str)
train.shape
train.head()

(78369, 126)

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_mean_cohtd,selling_price_std_cohtd,selling_price_last,other_discount_mean_cohtd,other_discount_std,other_discount_last,coupon_discount_mean_cohtd,coupon_discount_std,coupon_discount_last,coupon_applied_mean_cohtd,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_mean_cuhtd,selling_price_std_cuhtd,selling_price_last_cuhtd,other_discount_mean_cuhtd,other_discount_std_cuhtd,other_discount_last_cuhtd,coupon_discount_mean_cuhtd,coupon_discount_std_cuhtd,coupon_discount_last_cuhtd,coupon_applied_mean_cuhtd,coupon_applied_std_cuhtd,coupon_applied_last_cuhtd,selling_price_ewm_mean_cuhtd,selling_price_ewm_std_cuhtd,coupon_discount_ewm_mean_cuhtd,coupon_discount_ewm_std_cuhtd,coupon_applied_ewm_mean_cuhtd,coupon_applied_ewm_std_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_mean_cucohtd,selling_price_std_cucohtd,selling_price_last_cucohtd,other_discount_mean_cucohtd,other_discount_std_cucohtd,other_discount_last_cucohtd,coupon_discount_mean_cucohtd,coupon_discount_std_cucohtd,coupon_discount_last_cucohtd,coupon_applied_mean_cucohtd,coupon_applied_std_cucohtd,coupon_applied_last_cucohtd,selling_price_ewm_mean_cucohtd,selling_price_ewm_std_cucohtd,coupon_discount_ewm_mean_cucohtd,coupon_discount_ewm_std_cucohtd,coupon_applied_ewm_mean_cucohtd,coupon_applied_ewm_std_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,1,13,27,1053,0,X,2013-05-19,2013-07-05,47,3309.0,286278.21,86.515023,47.882148,207715.95,62.773028,4662.46,1.409024,158.0,0.047749,419.0,683.241551,539.584693,283.54,495.742124,561.417296,141.4,11.127589,37.025067,0.00,0.377088,0.795351,0.0,682.819199,404.264352,11.127188,26.336349,0.377079,0.587860,270,45754.09,169.459593,405.939171,7971.00,29.522222,0.00,0.000000,0,0.000000,73,626.768356,785.646696,362.61,109.191781,199.165691,43.46,0.000000,0.000000,0.0,0.000000,0.000000,0,626.095294,575.776022,0.000000,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,901.0,82671.50,91.755272,50.278461,51127.14,56.744883,2739.04,3.040000,70.0,0.077691,1662.0,217463.13,130.844242,177.121409,40888.55,24.602016,0.00,0.000000,0.0,0.000000,-1.0,-1.00,-1.000000,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0
1,2,13,116,48,0,X,2013-05-19,2013-07-05,47,6

In [11]:
print(group_col.head(5))
print(group_col_coupon.head(5))

0    13
1    13
2     9
3    13
4     8
Name: campaign_id, dtype: object
0      27
1     116
2     635
3     644
4    1017
Name: coupon_id, dtype: object


In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#rain[cat_cols].apply(le.fit_transform, inplace=True)
#tst[cat_cols].apply(le.fit_transform, inplace=True)
for col in test.columns.values:
       # Encoding only categorical variables
       if test[col].dtypes=='object' or test[col].dtypes=='datetime64[ns]':
       # Using whole data to form an exhaustive list of levels
           data=train[col].append(test[col])
           le.fit(data.values)
           train[col]=le.transform(train[col])
           test[col]=le.transform(test[col])

LabelEncoder()

LabelEncoder()

LabelEncoder()

In [13]:
test_ids = test[id_col]
train_ids = train[id_col]

In [14]:
y = train[target_col]
#droppping id and redemotion
train.drop(columns=[x for x in [id_col] + columns_to_drop + [target_col] if x in train.columns], inplace=True)
test.drop(columns=[x for x in [id_col] + columns_to_drop + [target_col] if x in test.columns], inplace=True)

In [15]:
print(test['end_date'].dtypes)

int64


In [16]:
train.shape
train.head()

(78369, 124)

,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date,duration,item_id_count,selling_price_sum,selling_price_mean,selling_price_std,other_discount_sum,other_discount_mean,coupon_discount_sum,coupon_discount_mean,coupon_applied_sum,coupon_applied_mean,date_count,selling_price_mean_cohtd,selling_price_std_cohtd,selling_price_last,other_discount_mean_cohtd,other_discount_std,other_discount_last,coupon_discount_mean_cohtd,coupon_discount_std,coupon_discount_last,coupon_applied_mean_cohtd,coupon_applied_std,coupon_applied_last,selling_price_ewm_mean,selling_price_ewm_std,coupon_discount_ewm_mean,coupon_discount_ewm_std,coupon_applied_ewm_mean,coupon_applied_ewm_std,item_id_count_cuht,selling_price_sum_cuht,selling_price_mean_cuht,selling_price_std_cuht,other_discount_sum_cuht,other_discount_mean_cuht,coupon_discount_sum_cuht,coupon_discount_mean_cuht,coupon_applied_sum_cuht,coupon_applied_mean_cuht,date_count_cuhtd,selling_price_mean_cuhtd,selling_price_std_cuhtd,selling_price_last_cuhtd,other_discount_mean_cuhtd,other_discount_std_cuhtd,other_discount_last_cuhtd,coupon_discount_mean_cuhtd,coupon_discount_std_cuhtd,coupon_discount_last_cuhtd,coupon_applied_mean_cuhtd,coupon_applied_std_cuhtd,coupon_applied_last_cuhtd,selling_price_ewm_mean_cuhtd,selling_price_ewm_std_cuhtd,coupon_discount_ewm_mean_cuhtd,coupon_discount_ewm_std_cuhtd,coupon_applied_ewm_mean_cuhtd,coupon_applied_ewm_std_cuhtd,item_id_count_cucoht,selling_price_sum_cucoht,selling_price_mean_cucoht,selling_price_std_cucoht,other_discount_sum_cucoht,other_discount_mean_cucoht,coupon_discount_sum_cucoht,coupon_discount_mean_cucoht,coupon_applied_sum_cucoht,coupon_applied_mean_cucoht,date_count_cucohtd,selling_price_mean_cucohtd,selling_price_std_cucohtd,selling_price_last_cucohtd,other_discount_mean_cucohtd,other_discount_std_cucohtd,other_discount_last_cucohtd,coupon_discount_mean_cucohtd,coupon_discount_std_cucohtd,coupon_discount_last_cucohtd,coupon_applied_mean_cucohtd,coupon_applied_std_cucohtd,coupon_applied_last_cucohtd,selling_price_ewm_mean_cucohtd,selling_price_ewm_std_cucohtd,coupon_discount_ewm_mean_cucohtd,coupon_discount_ewm_std_cucohtd,coupon_applied_ewm_mean_cucohtd,coupon_applied_ewm_std_cucohtd,item_id_count_cobrht,selling_price_sum_cobrht,selling_price_mean_cobrht,selling_price_std_cobrht,other_discount_sum_cobrht,other_discount_mean_cobrht,coupon_discount_sum_cobrht,coupon_discount_mean_cobrht,coupon_applied_sum_cobrht,coupon_applied_mean_cobrht,item_id_count_cubrht,selling_price_sum_cubrht,selling_price_mean_cubrht,selling_price_std_cubrht,other_discount_sum_cubrht,other_discount_mean_cubrht,coupon_discount_sum_cubrht,coupon_discount_mean_cubrht,coupon_applied_sum_cubrht,coupon_applied_mean_cubrht,item_id_count_cucobrht,selling_price_sum_cucobrht,selling_price_mean_cucobrht,selling_price_std_cucobrht,other_discount_sum_cucobrht,other_discount_mean_cucobrht,coupon_discount_sum_cucobrht,coupon_discount_mean_cucobrht,coupon_applied_sum_cucobrht,coupon_applied_mean_cucobrht
0,13,27,1053,0,16,16,47,3309.0,286278.21,86.515023,47.882148,207715.95,62.773028,4662.46,1.409024,158.0,0.047749,419.0,683.241551,539.584693,283.54,495.742124,561.417296,141.4,11.127589,37.025067,0.00,0.377088,0.795351,0.0,682.819199,404.264352,11.127188,26.336349,0.377079,0.587860,270,45754.09,169.459593,405.939171,7971.00,29.522222,0.00,0.000000,0,0.000000,73,626.768356,785.646696,362.61,109.191781,199.165691,43.46,0.000000,0.000000,0.0,0.000000,0.000000,0,626.095294,575.776022,0.000000,0.000000,0.000000,0.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,901.0,82671.50,91.755272,50.278461,51127.14,56.744883,2739.04,3.040000,70.0,0.077691,1662.0,217463.13,130.844242,177.121409,40888.55,24.602016,0.00,0.000000,0.0,0.000000,-1.0,-1.00,-1.000000,-1.000000,-1.00,-1.00,-1.0,-1.0,-1.0,-1.0
1,13,116,48,0,16,16,47,66.0,4717.27,71.473788,28.756222,677.15,10.259848,0.00,0.00000

Importing dependencies

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from time import time
import lightgbm as lgb
from catboost import CatBoostClassifier

XGBoost

In [18]:
speed={}
accuracy={}

model = XGBClassifier()
#model=GradientBoostingClassifier()

start=time()
cv=RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=0)
score=cross_val_score(model,train.values, y.values,scoring="accuracy",cv=cv,n_jobs=-1)

speed["GB"]=np.round(time() - start,3)
accuracy["GB"]=np.mean(score).round(3)

print(speed["GB"])
print(accuracy["GB"])
print(np.std(score))

49.271
0.992
0.0003495697642983282


In [19]:
xgbc = XGBClassifier()
xgbc.fit(train.values, y.values)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:25:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=True,
              validate_parameters=1, verbosity=None)

In [20]:
y_pred=xgbc.predict(test[train.columns].values)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [22]:
import joblib
# save model
joblib.dump(xgbc, 'xgbc.pkl')
# load model
#gbm_pickle = joblib.load('xgbc.pkl')

['xgbc.pkl']

In [23]:
pd.DataFrame({"id": train_ids, "redemption_status": xgbc}).to_csv('xgb-xgbc.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": y_pred}).to_csv('xgb-y-preds.csv', index=False)

LightGBM

In [24]:
speed={}
accuracy={}

model = lgb.LGBMClassifier()
#model=GradientBoostingClassifier()

start=time()
cv=RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=0)
score=cross_val_score(model,train.values, y.values,scoring="accuracy",cv=cv,n_jobs=-1)

speed["GB"]=np.round(time() - start,3)
accuracy["GB"]=np.mean(score).round(3)

print(speed["GB"])
print(accuracy["GB"])
print(np.std(score))

11.928
0.991
0.0003872107132188435


In [25]:
clf = lgb.LGBMClassifier()
clf.fit(train.values, y.values)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [26]:
y_pred=clf.predict(test[train.columns].values)

In [27]:
import joblib
# save model
joblib.dump(clf, 'lgb.pkl')
# load model
#gbm_pickle = joblib.load('lgb.pkl')

['lgb.pkl']

In [28]:
pd.DataFrame({"id": train_ids, "redemption_status": clf}).to_csv('lgb-clf.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": y_pred}).to_csv('lgb-y-preds.csv', index=False)

Catboost

In [29]:
speed={}
accuracy={}

model = CatBoostClassifier()
#model=GradientBoostingClassifier()

start=time()
cv=RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=0)
score=cross_val_score(model,train.values, y.values,scoring="accuracy",cv=cv,n_jobs=-1)

speed["GB"]=np.round(time() - start,3)
accuracy["GB"]=np.mean(score).round(3)

print(speed["GB"])
print(accuracy["GB"])
print(np.std(score))

239.195
0.992
0.00043016822475898104


In [30]:
model_cb = CatBoostClassifier()
model_cb.fit(train.values, y.values)

Learning rate set to 0.066334
0:	learn: 0.5296750	total: 103ms	remaining: 1m 42s
1:	learn: 0.4013976	total: 141ms	remaining: 1m 10s
2:	learn: 0.3074149	total: 179ms	remaining: 59.4s
3:	learn: 0.2392307	total: 216ms	remaining: 53.9s
4:	learn: 0.1902737	total: 254ms	remaining: 50.5s
5:	learn: 0.1549880	total: 288ms	remaining: 47.7s
6:	learn: 0.1264564	total: 327ms	remaining: 46.3s
7:	learn: 0.1059754	total: 365ms	remaining: 45.2s
8:	learn: 0.0902998	total: 410ms	remaining: 45.1s
9:	learn: 0.0778440	total: 452ms	remaining: 44.7s
10:	learn: 0.0684821	total: 497ms	remaining: 44.7s
11:	learn: 0.0617302	total: 533ms	remaining: 43.9s
12:	learn: 0.0567344	total: 568ms	remaining: 43.1s
13:	learn: 0.0515415	total: 608ms	remaining: 42.9s
14:	learn: 0.0483165	total: 649ms	remaining: 42.6s
15:	learn: 0.0454550	total: 688ms	remaining: 42.3s
16:	learn: 0.0430825	total: 725ms	remaining: 41.9s
17:	learn: 0.0413718	total: 761ms	remaining: 41.5s
18:	learn: 0.0399967	total: 797ms	remaining: 41.2s
19:	learn

162:	learn: 0.0243253	total: 5.92s	remaining: 30.4s
163:	learn: 0.0242759	total: 5.96s	remaining: 30.4s
164:	learn: 0.0242281	total: 6s	remaining: 30.4s
165:	learn: 0.0242027	total: 6.04s	remaining: 30.4s
166:	learn: 0.0241757	total: 6.08s	remaining: 30.3s
167:	learn: 0.0241334	total: 6.12s	remaining: 30.3s
168:	learn: 0.0241148	total: 6.15s	remaining: 30.3s
169:	learn: 0.0240284	total: 6.19s	remaining: 30.2s
170:	learn: 0.0239815	total: 6.23s	remaining: 30.2s
171:	learn: 0.0239537	total: 6.26s	remaining: 30.2s
172:	learn: 0.0239179	total: 6.3s	remaining: 30.1s
173:	learn: 0.0238930	total: 6.34s	remaining: 30.1s
174:	learn: 0.0238599	total: 6.38s	remaining: 30.1s
175:	learn: 0.0238413	total: 6.42s	remaining: 30.1s
176:	learn: 0.0238077	total: 6.46s	remaining: 30.1s
177:	learn: 0.0237824	total: 6.5s	remaining: 30s
178:	learn: 0.0237524	total: 6.55s	remaining: 30s
179:	learn: 0.0237091	total: 6.59s	remaining: 30s
180:	learn: 0.0236919	total: 6.63s	remaining: 30s
181:	learn: 0.0236535	tot

322:	learn: 0.0194126	total: 11.7s	remaining: 24.4s
323:	learn: 0.0193762	total: 11.7s	remaining: 24.4s
324:	learn: 0.0193437	total: 11.7s	remaining: 24.4s
325:	learn: 0.0193168	total: 11.8s	remaining: 24.3s
326:	learn: 0.0192936	total: 11.8s	remaining: 24.3s
327:	learn: 0.0192688	total: 11.8s	remaining: 24.2s
328:	learn: 0.0192283	total: 11.9s	remaining: 24.2s
329:	learn: 0.0192107	total: 11.9s	remaining: 24.2s
330:	learn: 0.0191724	total: 11.9s	remaining: 24.1s
331:	learn: 0.0191419	total: 12s	remaining: 24.1s
332:	learn: 0.0191138	total: 12s	remaining: 24.1s
333:	learn: 0.0190501	total: 12s	remaining: 24s
334:	learn: 0.0190444	total: 12.1s	remaining: 24s
335:	learn: 0.0190257	total: 12.1s	remaining: 23.9s
336:	learn: 0.0189857	total: 12.1s	remaining: 23.9s
337:	learn: 0.0189602	total: 12.2s	remaining: 23.9s
338:	learn: 0.0189352	total: 12.2s	remaining: 23.8s
339:	learn: 0.0189080	total: 12.3s	remaining: 23.8s
340:	learn: 0.0188994	total: 12.3s	remaining: 23.8s
341:	learn: 0.0188925	

481:	learn: 0.0158259	total: 17s	remaining: 18.3s
482:	learn: 0.0158054	total: 17s	remaining: 18.2s
483:	learn: 0.0157773	total: 17.1s	remaining: 18.2s
484:	learn: 0.0157684	total: 17.1s	remaining: 18.2s
485:	learn: 0.0157473	total: 17.1s	remaining: 18.1s
486:	learn: 0.0157231	total: 17.2s	remaining: 18.1s
487:	learn: 0.0156939	total: 17.2s	remaining: 18s
488:	learn: 0.0156634	total: 17.2s	remaining: 18s
489:	learn: 0.0156334	total: 17.3s	remaining: 18s
490:	learn: 0.0156067	total: 17.3s	remaining: 17.9s
491:	learn: 0.0155932	total: 17.3s	remaining: 17.9s
492:	learn: 0.0155850	total: 17.4s	remaining: 17.9s
493:	learn: 0.0155656	total: 17.4s	remaining: 17.8s
494:	learn: 0.0155385	total: 17.4s	remaining: 17.8s
495:	learn: 0.0155303	total: 17.5s	remaining: 17.8s
496:	learn: 0.0155271	total: 17.5s	remaining: 17.7s
497:	learn: 0.0155048	total: 17.5s	remaining: 17.7s
498:	learn: 0.0154674	total: 17.6s	remaining: 17.6s
499:	learn: 0.0154412	total: 17.6s	remaining: 17.6s
500:	learn: 0.0154136	

642:	learn: 0.0131470	total: 22.3s	remaining: 12.4s
643:	learn: 0.0131214	total: 22.3s	remaining: 12.3s
644:	learn: 0.0131152	total: 22.3s	remaining: 12.3s
645:	learn: 0.0131036	total: 22.4s	remaining: 12.3s
646:	learn: 0.0130844	total: 22.4s	remaining: 12.2s
647:	learn: 0.0130624	total: 22.4s	remaining: 12.2s
648:	learn: 0.0130502	total: 22.5s	remaining: 12.1s
649:	learn: 0.0130189	total: 22.5s	remaining: 12.1s
650:	learn: 0.0130127	total: 22.5s	remaining: 12.1s
651:	learn: 0.0129921	total: 22.6s	remaining: 12s
652:	learn: 0.0129839	total: 22.6s	remaining: 12s
653:	learn: 0.0129653	total: 22.6s	remaining: 12s
654:	learn: 0.0129480	total: 22.7s	remaining: 11.9s
655:	learn: 0.0129368	total: 22.7s	remaining: 11.9s
656:	learn: 0.0129263	total: 22.7s	remaining: 11.9s
657:	learn: 0.0129039	total: 22.8s	remaining: 11.8s
658:	learn: 0.0128911	total: 22.8s	remaining: 11.8s
659:	learn: 0.0128812	total: 22.8s	remaining: 11.8s
660:	learn: 0.0128582	total: 22.9s	remaining: 11.7s
661:	learn: 0.0128

805:	learn: 0.0110503	total: 27.8s	remaining: 6.69s
806:	learn: 0.0110335	total: 27.8s	remaining: 6.66s
807:	learn: 0.0110176	total: 27.9s	remaining: 6.62s
808:	learn: 0.0110121	total: 27.9s	remaining: 6.59s
809:	learn: 0.0109887	total: 27.9s	remaining: 6.55s
810:	learn: 0.0109853	total: 28s	remaining: 6.52s
811:	learn: 0.0109725	total: 28s	remaining: 6.48s
812:	learn: 0.0109704	total: 28s	remaining: 6.45s
813:	learn: 0.0109591	total: 28.1s	remaining: 6.42s
814:	learn: 0.0109494	total: 28.1s	remaining: 6.38s
815:	learn: 0.0109385	total: 28.1s	remaining: 6.35s
816:	learn: 0.0109149	total: 28.2s	remaining: 6.31s
817:	learn: 0.0109071	total: 28.2s	remaining: 6.28s
818:	learn: 0.0108884	total: 28.3s	remaining: 6.25s
819:	learn: 0.0108822	total: 28.3s	remaining: 6.21s
820:	learn: 0.0108721	total: 28.3s	remaining: 6.18s
821:	learn: 0.0108685	total: 28.4s	remaining: 6.14s
822:	learn: 0.0108544	total: 28.4s	remaining: 6.11s
823:	learn: 0.0108506	total: 28.4s	remaining: 6.07s
824:	learn: 0.0108

966:	learn: 0.0094993	total: 33.3s	remaining: 1.14s
967:	learn: 0.0094985	total: 33.3s	remaining: 1.1s
968:	learn: 0.0094948	total: 33.3s	remaining: 1.07s
969:	learn: 0.0094878	total: 33.4s	remaining: 1.03s
970:	learn: 0.0094761	total: 33.4s	remaining: 998ms
971:	learn: 0.0094574	total: 33.4s	remaining: 963ms
972:	learn: 0.0094471	total: 33.5s	remaining: 929ms
973:	learn: 0.0094375	total: 33.5s	remaining: 894ms
974:	learn: 0.0094306	total: 33.5s	remaining: 860ms
975:	learn: 0.0094252	total: 33.6s	remaining: 825ms
976:	learn: 0.0094226	total: 33.6s	remaining: 791ms
977:	learn: 0.0094204	total: 33.6s	remaining: 756ms
978:	learn: 0.0094138	total: 33.7s	remaining: 722ms
979:	learn: 0.0094054	total: 33.7s	remaining: 688ms
980:	learn: 0.0093973	total: 33.7s	remaining: 653ms
981:	learn: 0.0093891	total: 33.8s	remaining: 619ms
982:	learn: 0.0093778	total: 33.8s	remaining: 584ms
983:	learn: 0.0093774	total: 33.8s	remaining: 550ms
984:	learn: 0.0093696	total: 33.9s	remaining: 516ms
985:	learn: 0

In [31]:
y_pred=model_cb.predict(test[train.columns].values)

In [32]:
import joblib
# save model
joblib.dump(model_cb, 'cb.pkl')
# load model
#gbm_pickle = joblib.load('cb.pkl')

['cb.pkl']

In [33]:
pd.DataFrame({"id": train_ids, "redemption_status": model_cb}).to_csv('cb-model_cb.csv', index=False)
pd.DataFrame({"id": test_ids, "redemption_status": y_pred}).to_csv('cb-y-preds.csv', index=False)